## This notebook is used for testing M1M3 communication with the pointing component

Step-by-step description of what we do can be found in 
Jira test case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2173

In summary:

It check the M1M3 state transitions and force commands

It also checks the LUT against the input polynomials, before and after a slew

In [ ]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts.idl.enums import MTM1M3
from M1M3tools import *

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

plt.jet();

In [ ]:
#summit = 1 #use this for summit testing
summit = 0 #use this for NCSA

In [ ]:
#Tiago suggested not to run this on the summit; this is only for NCSA debugging DDS
if not summit:
    import os
    print(os.environ["OSPL_URI"])
    if os.environ.get("LSST_DDS_ALIGNER", "false") != "false":
        print("LSST_DDS_ALIGNER is mis-configured")

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
if not summit:
    await asyncio.sleep(10) #not need to wait on summit since we use the daemon
    ptg = salobj.Remote(script.domain, "MTPtg")
    rot = salobj.Remote(script.domain, "MTRotator")
mount = salobj.Remote(script.domain, "MTMount") #we still need it on the summit, to check fake mount telemetry!
m1m3 = salobj.Remote(script.domain, "MTM1M3", exclude=['logMessage'])
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
if summit:
    await m1m3.start_task
else:
    await asyncio.gather(ptg.start_task,
                         mount.start_task,
                         script.start_task, #                     
                         rot.start_task,
                         m1m3.start_task)

In [ ]:
await m1m3.evt_heartbeat.next(flush=True, timeout=5)

### test state transition. If m1m3 is already enabled, disable then enable it.

In [ ]:
sstate = await m1m3.evt_summaryState.aget(timeout=5)
print('staring with: m1m3 state',salobj.State(sstate.summaryState), pd.to_datetime(sstate.private_sndStamp, unit='s'))
dstate = await m1m3.evt_detailedState.aget(timeout=200)
print('m1m3 state', MTM1M3.DetailedState(dstate.detailedState), pd.to_datetime(dstate.private_sndStamp, unit='s'))

In [ ]:
async def lowerMirror(m1m3):
    m1m3.evt_detailedState.flush()
    await m1m3.cmd_lowerM1M3.set_start(lowerM1M3=True, timeout = 30)
    while True:
        state = await m1m3.evt_detailedState.next(flush=False, timeout=300)
        print('m1m3 state', MTM1M3.DetailedState(state.detailedState), pd.to_datetime(state.private_sndStamp, unit='s'))
        if (MTM1M3.DetailedState(state.detailedState) == MTM1M3.DetailedState.PARKED
                or MTM1M3.DetailedState(state.detailedState) == MTM1M3.DetailedState.PARKEDENGINEERING):
            break

async def raiseMirror(m1m3):
    m1m3.evt_detailedState.flush()
    await m1m3.cmd_raiseM1M3.set_start(raiseM1M3=True, timeout = 30)
    while True:
        state = await m1m3.evt_detailedState.next(flush=False, timeout=300)
        print('m1m3 state', MTM1M3.DetailedState(state.detailedState), pd.to_datetime(state.private_sndStamp, unit='s'))
        if (MTM1M3.DetailedState(state.detailedState) == MTM1M3.DetailedState.ACTIVE
                or MTM1M3.DetailedState(state.detailedState) == MTM1M3.DetailedState.ACTIVEENGINEERING):
            break
        
if (MTM1M3.DetailedState(dstate.detailedState) == MTM1M3.DetailedState.ACTIVEENGINEERING
        or MTM1M3.DetailedState(dstate.detailedState) == MTM1M3.DetailedState.ACTIVE):
    await lowerMirror(m1m3)

In [ ]:
if sstate.summaryState == salobj.State.ENABLED:
    await salobj.set_summary_state(m1m3, salobj.State.DISABLED) #disable m1m3
if sstate.summaryState == salobj.State.FAULT:
    await m1m3.cmd_clearErrors.set_start()
if sstate.summaryState == salobj.State.STANDBY:
    await m1m3.cmd_start.set_start(settingsToApply="Default")
await salobj.set_summary_state(m1m3, salobj.State.ENABLED) #enable m1m3

In [ ]:
#may need to wait a few seconds before event shows up in EFD
await asyncio.sleep(10.)
from lsst_efd_client import EfdClient

if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1
end = Time(datetime.now(), scale='tai')
start = end - timedelta(hours=1)
dfe = await client.select_time_series('lsst.sal.MTM1M3.logevent_summaryState', '*', start, end, csc_index)
#to check messages in Kafka, go to https://lsst-kafka-0-nts-efd.ncsa.illinois.edu/
##we cannot get time series data from DDS. We have to query the EFD

In [ ]:
dfe.tail(5)

### Raise M1M3

In [ ]:
m1m3Angle = await m1m3.tel_inclinometerData.aget()
print("m1m3Angle", m1m3Angle.inclinometerAngle)

In [ ]:
m1m3HP = await m1m3.tel_hardpointActuatorData.aget()
print("m1m3 z", m1m3HP.zPosition)

In [ ]:
await raiseMirror(m1m3)

In [ ]:
#if you want to examine the test we did on the summit, you need to go to the Jira test execution and grap the timestamps
end = Time(datetime.now()) #- timedelta(minutes=75)
#end = Time('2021-02-04T22:50:00')
#end = Time('2021-02-12T22:24:00')
start = end - timedelta(minutes=10)
dfz = await client.select_time_series('lsst.sal.MTM1M3.hardpointActuatorData', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
dfr = await client.select_time_series('lsst.sal.MTM1M3.command_raiseM1M3', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
dfl = await client.select_time_series('lsst.sal.MTM1M3.command_lowerM1M3', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
dfe = await client.select_time_series('lsst.sal.MTM1M3.logevent_summaryState', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)
dfd = await client.select_time_series('lsst.sal.MTM1M3.logevent_detailedState', '*', 
                                     (start-timedelta(seconds=37)).tai, (end-timedelta(seconds=37)).tai, csc_index)

In [ ]:
print(len(dfz), len(dfr), len(dfl))

In [ ]:
dfr

In [ ]:
zFactor = 1000 #we magnify the zPosition (unit = micron) by this factor to make it visible on the plot below
fig, ax = plt.subplots(figsize=(15,8))
plt.plot(dfz.zPosition*zFactor, label='zPosition (mm)')
plt.plot(dfr.raiseM1M3, 'o', label='raise command', markersize=20)
if len(dfl)>0:
    plt.plot(dfl.lowerM1M3, 'v', label='lower command', markersize=20)
plt.plot(dfe.summaryState, '*', label='summary state', markersize=20)
plt.plot(dfd.detailedState, 'x', label='detailed state', markersize=20)
plt.legend(fontsize=15)
plt.grid();

In [ ]:
for i in range(1,13):
    print(i, MTM1M3.DetailedState(i))

### check the forces

In [ ]:
#If magnitude = 0.0 then it is off
m1m3ForceBalance = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
print("starting with magnitude of the m1m3 balance force ---", m1m3ForceBalance.forceMagnitude, "----",
      pd.to_datetime(m1m3ForceBalance.private_sndStamp, unit='s'))

In [ ]:
m1m3ForceBalance = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
if not m1m3ForceBalance.forceMagnitude:
    await m1m3.cmd_enableHardpointCorrections.set_start(timeout=10)
    await asyncio.sleep(3.)
    m1m3ForceBalance = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
    print("Magnitude of the m1m3 force balance system", m1m3ForceBalance.forceMagnitude)

In [ ]:
fx = [0]*12
fy = [0]*100
fz = [0]*156
#offsetForces not valid in ActiveState; Petr is working on enabling this for summit as well
#await m1m3.cmd_applyOffsetForces.set_start(xForces=fx, yForces=fy, zForces=fz) 
await m1m3.cmd_applyAberrationForces.set_start(zForces=fz) 
await m1m3.cmd_applyActiveOpticForces.set_start(zForces=fz)

In [ ]:
fel = await m1m3.evt_appliedElevationForces.aget(timeout=10.)
faz = await m1m3.evt_appliedAzimuthForces.aget(timeout=10.)
fth = await m1m3.evt_appliedThermalForces.aget(timeout=10.)
fba = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
fac = await m1m3.evt_appliedAccelerationForces.aget(timeout=10.)
fve = await m1m3.evt_appliedVelocityForces.aget(timeout=10.)
fst = await m1m3.evt_appliedStaticForces.aget(timeout=10.)
fab = await m1m3.evt_appliedAberrationForces.aget(timeout=10.)
fof = await m1m3.evt_appliedOffsetForces.aget(timeout=10.)
fao = await m1m3.evt_appliedActiveOpticForces.aget(timeout=10.)
fapp = await m1m3.evt_appliedForces.aget(timeout=10.)

In [ ]:
ftel = await m1m3.tel_forceActuatorData.aget(timeout=10.)

In [ ]:
def plotForces(fel, fba, fst, fao):
    fig, ax = plt.subplots(3,1, figsize=(15,8))
    ax[0].plot(fel.xForces, '-o', label='elevation');
    ax[0].plot(fba.xForces, label='FB')
    ax[0].plot(fst.xForces, label='static')
    ax[0].plot(ftel.xForce, '-v', label='measured')
    ax[0].legend()
    ax[0].set_title('XForces')
    ax[1].plot(fel.yForces, '-o', label='elevation');
    #ax[1].plot(fba.yForces, label='FB')
    #ax[1].plot(fst.yForces, label='static')
    ax[1].plot(ftel.yForce, '-v', label='measured')
    ax[1].legend()
    ax[1].set_title('YForces')
    ax[2].plot(fel.zForces, '-o', label='elevation');
    ax[2].plot(fba.zForces, label='FB')
    ax[2].plot(fst.zForces, label='static')
    ax[2].plot(fao.zForces, label='AOS')
    ax[2].plot(ftel.zForce, '-v', label='measured')
    ax[2].set_title('ZForces')
    ax[2].legend()
    
    fig2, ax=plt.subplots( 1,3, figsize = [15,4])
    aa = np.array(fao.zForces)
    img = ax[0].scatter(xact, yact, c=aa, s=abs(aa)*2)
    #plt.jet()
    ax[0].axis('equal')
    ax[0].set_title('AOS forces')
    fig.colorbar(img, ax=ax[0])

    aa = np.array(fel.zForces)
    img = ax[1].scatter(xact, yact, c=aa, s=abs(aa)*0.1)
    #plt.jet()
    ax[1].axis('equal')
    ax[1].set_title('elevation forces')
    fig.colorbar(img, ax=ax[1])
    
    aa = np.array(fst.zForces)
    img = ax[2].scatter(xact, yact, c=aa, s=abs(aa)*10)
    #plt.jet()
    ax[2].axis('equal')
    ax[2].set_title('static forces')
    fig.colorbar(img, ax=ax[2])

In [ ]:
plotForces(fel, fba, fst, fao)

apply some nonzero forces

In [ ]:
fx = [2]*12
fy = [3]*100
fz = [-5]*156
#offsetForces not valid in ActiveState; Petr is working on enabling this for summit as well
#await m1m3.cmd_applyOffsetForces.set_start(xForces=fx, yForces=fy, zForces=fz)
await m1m3.cmd_applyAberrationForces.set_start(zForces=fz)
await m1m3.cmd_applyActiveOpticForces.set_start(zForces=fz)

In [ ]:
fel = await m1m3.evt_appliedElevationForces.aget(timeout=10.)
faz = await m1m3.evt_appliedAzimuthForces.aget(timeout=10.)
fth = await m1m3.evt_appliedThermalForces.aget(timeout=10.)
fba = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
fac = await m1m3.evt_appliedAccelerationForces.aget(timeout=10.)
fve = await m1m3.evt_appliedVelocityForces.aget(timeout=10.)
fst = await m1m3.evt_appliedStaticForces.aget(timeout=10.)
fab = await m1m3.evt_appliedAberrationForces.aget(timeout=10.)
fof = await m1m3.evt_appliedOffsetForces.aget(timeout=10.)
fao = await m1m3.evt_appliedActiveOpticForces.aget(timeout=10.)
fapp = await m1m3.evt_appliedForces.aget(timeout=10.)
ftel = await m1m3.tel_forceActuatorData.aget(timeout=10.)
plotForces(fel, fba, fst, fao)

In [ ]:
max(abs(np.array(fao.zForces)))

### Check the forces add up as expected

In [ ]:
xyz = 'xyz'
nForce = [12, 100, 156]
for i in range(3):
    print('%s diff'%xyz[i], end=" ")
    for j in range(nForce[i]):
        diff = \
        getattr(fel, '%sForces'%(xyz[i]))[j] + getattr(faz, '%sForces'%(xyz[i]))[j] + getattr(fth, '%sForces'%(xyz[i]))[j] + \
        getattr(fba, '%sForces'%(xyz[i]))[j] + getattr(fac, '%sForces'%(xyz[i]))[j] + getattr(fve, '%sForces'%(xyz[i]))[j] + \
        getattr(fst, '%sForces'%(xyz[i]))[j] + getattr(fof, '%sForces'%(xyz[i]))[j]  \
           - getattr(ftel, '%sForce'%(xyz[i]))[j] # - getattr(fapp, '%sForces'%(xyz[i]))[j] #
        if i==2: #only for z forces
            diff = diff + getattr(fao, '%sForces'%(xyz[i]))[j] + getattr(fab, '%sForces'%(xyz[i]))[j] 
        print('%d, %.1f'%(j,diff), end=" | ")
        if abs(diff)>1:
            print('\nfel faz fth fba fac, fve fst fof ftel, (fao fab) ')
            print('%d -------------------------------------------------'%j)
            print('%.1f, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f, %.1f, '%(
                getattr(fel, '%sForces'%(xyz[i]))[j], getattr(faz, '%sForces'%(xyz[i]))[j], getattr(fth, '%sForces'%(xyz[i]))[j],
                getattr(fba, '%sForces'%(xyz[i]))[j], getattr(fac, '%sForces'%(xyz[i]))[j], getattr(fve, '%sForces'%(xyz[i]))[j],
                getattr(fst, '%sForces'%(xyz[i]))[j], getattr(fof, '%sForces'%(xyz[i]))[j],
                getattr(ftel, '%sForce'%(xyz[i]))[j] ), end="")
            if i==2:
                print('%.1f, %.1f,'%(getattr(fao, '%sForces'%(xyz[i]))[j], getattr(fab, '%sForces'%(xyz[i]))[j]))
            break
    print('\n')

### check the force components against expectations

In [ ]:
m1m3Angle = await m1m3.tel_inclinometerData.aget()
eA = m1m3Angle.inclinometerAngle #elevation angle
print("m1m3 elevation Angle", eA, pd.to_datetime(m1m3Angle.private_sndStamp, unit='s'))

In [ ]:
lutFile = '%s/notebooks/ts_m1m3support/SettingFiles/Tables/ElevationZTable.csv'%os.environ['HOME']
lutel = pd.read_csv(lutFile)

In [ ]:
def lookUpElevation(lute, zA):
    '''
    the input to the LUT is zenith angle
    '''
    eForces = np.zeros(156)
    for i in range(156):
        coeff = [lute['Coefficient %d'%j][i] for j in range(5,-1,-1)]
        mypoly = np.poly1d(coeff)
        eForces[i] = mypoly(zA)
    return eForces
eForces = lookUpElevation(lutel, 90-eA) 

In [ ]:
dAngle = 0.3 #deg

In [ ]:
# if the predicted change in LUT forces is not satisfactory, go back and change dAngle. 
# Around zenith pointing, we found each 0.1 deg gives max force difference of ~1N.
eForces1 = lookUpElevation(lutel, 90-eA+dAngle)
fig, ax = plt.subplots(1,2, figsize=(15,5))
ax[0].plot(eForces, label = 'expected')
ax[0].plot([fel.zForces[i] for i in range(156)], '--', label='EFD')
ax[0].legend()
ax[0].set_title('Elevation Z Forces')

ax[1].plot(fel.zForces - eForces, label='measured-expected')
ax[1].plot(eForces1 - eForces, label='force change due to %.1f deg change in angle'%dAngle)
ax[1].legend();

In [ ]:
lutFile = '%s/notebooks/ts_m1m3support/SettingFiles/Tables/StaticZTable.csv'%os.environ['HOME']
lutst = pd.read_csv(lutFile)

In [ ]:
plt.plot(lutst['ZForce'], label = 'expected')
plt.plot([fst.zForces[i] for i in range(156)], label='EFD')
plt.legend()
plt.title('Static Z Forces');

## Do a slew, then check the LUT again (only for NCSA testing)
We can either do this by commanding the mount using the ptg, or we can create a Controller object to send mount telemetry.

In [ ]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
alt = 40 * u.deg
az = 0 * u.deg

target_name="TMA motion test"
time_data = await ptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_ptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_ptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

print(curr_time_ptg.tai.value)

cmd_elaz = AltAz(alt=alt, az=az, 
                obstime=curr_time_ptg.tai, 
                location=mtcs.location)
cmd_radec = cmd_elaz.transform_to(ICRS)
await mtcs.slew_icrs(ra=cmd_radec.ra, dec=cmd_radec.dec, rot=0., rot_type=RotType.PhysicalSky)

In [ ]:
if not summit:
    mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
    rotStatus = await rot.evt_inPosition.aget(timeout=5.)
    print('Are we tracking?', mountStatus.elevation , mountStatus.azimuth , rotStatus.inPosition)

In [ ]:
if not summit:
    await ptg.cmd_stopTracking.set_start(timeout=5.)

#### check angle and LUT after the slew

In [ ]:
if not summit:
    mountAngle = await mount.tel_elevation.aget(timeout=10.)
    print("mount elevation angle", mountAngle.actualPosition)
    elev = mountAngle.actualPosition

In [ ]:
if not summit:
    m1m3Angle = await m1m3.tel_inclinometerData.aget()
    eA = m1m3Angle.inclinometerAngle
    print("m1m3 elevation Angle", eA)

    eForces = lookUpElevation(lutel, 90-eA)

## create a Controller object to send mount telemetry (for NCSA OR summit)

#### If we are to use the Controller object, we need to take the mount out of the disabled/enabled states so that it stops sending telemetry data.
Get fake telemetry ready first (using mountTelGenerator.ipynb)

Then we need to point m1m3 to the fake mount elevation

In [ ]:
#take m1m3 to standby
await lowerMirror(m1m3)
await salobj.set_summary_state(m1m3, salobj.State.STANDBY)

In [ ]:
#if not summit:
#     on a terminal, log onto the teststand, enter the container, edit the config so that M1M3 uses the mount telemetry for its LUT
#     [bxin@lsst-teststand-ts2 MT]$ docker exec -it mtm1m3_sim bash
# else:
#    ask Petr to do this.

#[saluser@4fc00c04ac94 ~]$ vi ts_m1m3support/SettingFiles/Sets/Default/1/ForceActuatorSettings.xml
# we need: <UseInclinometer>0</UseInclinometer>

In [ ]:
await salobj.set_summary_state(m1m3, salobj.State.ENABLED,  settingsToApply='Default')

In [ ]:
await raiseMirror(m1m3)

In [ ]:
# only in case that we need to abort the raise
# await m1m3.cmd_abortRaiseM1M3.set_start(timeout = 30)

In [ ]:
m1m3ForceBalance = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
if not m1m3ForceBalance.forceMagnitude:
    await m1m3.cmd_enableHardpointCorrections.set_start(timeout=10)
    await asyncio.sleep(3.)
    m1m3ForceBalance = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
    print("Magnitude of the m1m3 force balance system", m1m3ForceBalance.forceMagnitude)

In [ ]:
mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.angleActual, pd.to_datetime(mountAngle.private_sndStamp, unit='s'))

In [ ]:
fel2 = await m1m3.evt_appliedElevationForces.aget(timeout=10.)
faz2 = await m1m3.evt_appliedAzimuthForces.aget(timeout=10.)
fth2 = await m1m3.evt_appliedThermalForces.aget(timeout=10.)
fba2 = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
fac2 = await m1m3.evt_appliedAccelerationForces.aget(timeout=10.)
fve2 = await m1m3.evt_appliedVelocityForces.aget(timeout=10.)
fst2 = await m1m3.evt_appliedStaticForces.aget(timeout=10.)
fab2 = await m1m3.evt_appliedAberrationForces.aget(timeout=10.)
fof2 = await m1m3.evt_appliedOffsetForces.aget(timeout=10.)
fao2 = await m1m3.evt_appliedActiveOpticForces.aget(timeout=10.)
ftel2 = await m1m3.tel_forceActuatorData.aget(timeout=10.)
eForces2 = lookUpElevation(lutel, 90 - mountAngle.angleActual) #Input to LUT is zenith angle, need to change!

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,5))
ax[0].plot([fel.zForces[i] for i in range(156)] - eForces, '-', label='EFD - expected')
ax[0].plot([fel2.zForces[i] for i in range(156)] - eForces2, '-o', label='EFD2 - expected2')
ax[0].legend()

ax[1].plot(eForces - eForces2, label='expected2-expected')
ax[1].legend();

In [ ]:
#check the change in the balance forces
plotForces(fel2, fba2, fst2, fao2)

In [ ]:
plt.plot(fba.zForces, label='before')
plt.plot(fba2.zForces, label='now')
plt.grid()
plt.title('Balanced forces - z')
plt.legend();

#### repeat the above using 89.5 deg elevation angle

In [ ]:
#need to go to mountTelGenerator.ipynb, stop the telemetry, and resend as 89.5 deg
mountAngle = await mount.tel_elevation.aget(timeout=10.)
print("mount elevation angle", mountAngle.angleActual, pd.to_datetime(mountAngle.private_sndStamp, unit='s'))

In [ ]:
fel3 = await m1m3.evt_appliedElevationForces.aget(timeout=10.)
faz3 = await m1m3.evt_appliedAzimuthForces.aget(timeout=10.)
fth3 = await m1m3.evt_appliedThermalForces.aget(timeout=10.)
fba3 = await m1m3.evt_appliedBalanceForces.aget(timeout=10.)
fac3 = await m1m3.evt_appliedAccelerationForces.aget(timeout=10.)
fve3 = await m1m3.evt_appliedVelocityForces.aget(timeout=10.)
fst3 = await m1m3.evt_appliedStaticForces.aget(timeout=10.)
fab3 = await m1m3.evt_appliedAberrationForces.aget(timeout=10.)
fof3 = await m1m3.evt_appliedOffsetForces.aget(timeout=10.)
fao3 = await m1m3.evt_appliedActiveOpticForces.aget(timeout=10.)
ftel3 = await m1m3.tel_forceActuatorData.aget(timeout=10.)
eForces3 = lookUpElevation(lutel, 90-mountAngle.angleActual)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,5))
ax[0].plot([fel.zForces[i] for i in range(156)] - eForces, '-', label='EFD - expected')
ax[0].plot([fel2.zForces[i] for i in range(156)] - eForces2, '-o', label='EFD2 - expected2')
ax[0].plot([fel3.zForces[i] for i in range(156)] - eForces3, '-', label='EFD3 - expected3')
ax[0].legend()

ax[1].plot(eForces - eForces2, label='expected2-expected')
ax[1].plot(eForces - eForces3, label='expected3-expected')
ax[1].grid()
ax[1].legend();

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,10))
plt.plot(fba.zForces, label='90 deg')
plt.plot(fba2.zForces, label='89.7 deg')
plt.plot(fba3.zForces, 'o', label='now (89.5 deg)')
plt.grid()
plt.title('Balanced forces - z')
plt.legend();

### Close up

In [ ]:
#if we started with enabled state, we need to put it back
await salobj.set_summary_state(m1m3, salobj.State.ENABLED,  settingsToApply='Default')

In [ ]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)

### Check if the telescope is in tracking mode. If yes, need to stop stacking. 
The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await ptg.cmd_stopTracking.set_start(timeout=5.)